In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Simplified data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)
    

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 30  # Keeping a higher number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 800 files belonging to 20 classes.
Found 200 files belonging to 20 classes.
Epoch 1/30


KeyboardInterrupt: 

In [5]:
# model.save('model3.h5')

In [9]:
# model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('model3.h5')

# Define function to preprocess an image file for prediction
def preprocess_image(file_path):
    img = image.load_img(file_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Example usage: Predicting on a single image
image_path = 'dataset/uji/rawon2.jpg'
preprocessed_img = preprocess_image(image_path)
predictions = model.predict(preprocessed_img)
predicted_class = np.argmax(predictions[0])
confidence = predictions[0][predicted_class]

# Retrieve class labels
class_names_file = 'labels.txt'
with open(class_names_file, 'r') as f:
    class_names = f.read().splitlines()

# Print results
print(f'Predicted class: {class_names[predicted_class]}')
print(f'Confidence: {confidence:.2f}')

1/1 [==============================] - 1s 836ms/step
Predicted class: Rawon: 400-500 kalori
Confidence: 1.00


In [3]:
!tensorflowjs_converter --input_format=keras ./model3.h5 ./

2024-06-18 14:07:18.209946: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

Traceback (most recent call last):
  File "C:\Users\david\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\david\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\david\AppData\Local\Programs\Python\Python39\Scripts\tensorflowjs_converter.exe\__main__.py", line 4, in <module>
  File "C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflowjs\__init__.py", line 21, in <module>
    from tensorflowjs import converters
  File "C:\Users\david\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorf

In [7]:
import tensorflow_decision_forests as tfdf

NotFoundError: c:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_decision_forests\tensorflow\ops\inference\inference.so not found

In [7]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Read class names from file

class_names_file = 'labels.txt'
with open(class_names_file, 'r') as f:
    class_names = f.read().splitlines()
    

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path='model3_efficientnet.tflite')
interpreter.allocate_tensors()

# Get input and output details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image.
def preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path)
    img = img.resize(target_size)
    img = np.asarray(img)
    img = img.astype('float32')
    img /= 255.0  # Normalize to [0, 1]
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Example usage:
image_path = 'dataset/uji/soto1.jpg'
input_image = preprocess_image(image_path)

# Check input shape
input_shape = input_details[0]['shape']
if input_image.shape != tuple(input_shape):
    raise ValueError(f"Input image shape {input_image.shape} doesn't match expected shape {input_shape}")

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_image)

# Run inference
interpreter.invoke()

# Get output tensor
output = interpreter.get_tensor(output_details[0]['index'])

# Interpret the output (assuming softmax probabilities)
predicted_class_index = np.argmax(output)
predicted_class = class_names[predicted_class_index]  # Assuming class_names is defined earlier
confidence = output[0][predicted_class_index]  # Confidence score for the predicted class

print(f'Predicted class: {predicted_class}')
print(f'Confidence: {confidence}')

Predicted class: Rendang: 450-550 kalori
Confidence: 0.06813720613718033
